In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report

In [2]:
VOCAB_SIZE = 20000
EMBED_DIM = 128
RNN_SIZE = 128
CLIP_NORM = 5.0
BATCH_SIZE = 32
DISPLAY_STEP = 50
N_EPOCH = 2
N_CLASS = 2

In [3]:
def sort_by_len(x, y):
    idx = sorted(range(len(x)), key=lambda i: len(x[i]))
    return x[idx], y[idx]

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)
X_train, y_train = sort_by_len(X_train, y_train)
X_test, y_test = sort_by_len(X_test, y_test)

In [4]:
def pad_sentence_batch(sent_batch):
    max_seq_len = max([len(sent) for sent in sent_batch])
    padded_seqs = [(sent + [0]*(max_seq_len - len(sent))) for sent in sent_batch]
    return padded_seqs

def next_train_batch():
    for i in range(0, len(X_train), BATCH_SIZE):
        padded_seqs = pad_sentence_batch(X_train[i : i+BATCH_SIZE])
        yield padded_seqs, y_train[i : i+BATCH_SIZE]
        
def next_test_batch():
    for i in range(0, len(X_test), BATCH_SIZE):
        padded_seqs = pad_sentence_batch(X_test[i : i+BATCH_SIZE])
        yield padded_seqs
        
def pipeline(mode):
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = tf.data.Dataset.from_generator(next_train_batch, (tf.int32,tf.int64),
            (tf.TensorShape([None,None]),tf.TensorShape([None])))
        dataset = dataset.repeat(N_EPOCH)
    if mode == tf.estimator.ModeKeys.PREDICT:
        dataset = tf.data.Dataset.from_generator(next_test_batch, tf.int32,
            tf.TensorShape([None,None]))
    iterator = dataset.make_one_shot_iterator()
    return iterator

In [5]:
def rnn_cell():
    return tf.nn.rnn_cell.GRUCell(RNN_SIZE//2, kernel_initializer=tf.orthogonal_initializer())

def forward(inputs, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(inputs, VOCAB_SIZE, EMBED_DIM)
        x = tf.layers.dropout(x, 0.2, training=is_training)
        _, bi_states = tf.nn.bidirectional_dynamic_rnn(
            rnn_cell(), rnn_cell(), x, tf.count_nonzero(inputs, 1), dtype=tf.float32)
        x = tf.concat(bi_states, -1)
        logits = tf.layers.dense(x, N_CLASS)
    return logits

def clip_grads(loss):
    params = tf.trainable_variables()
    grads = tf.gradients(loss, params)
    clipped_grads, _ = tf.clip_by_global_norm(grads, CLIP_NORM)
    return zip(clipped_grads, params)

In [6]:
ops = {}

X_train_batch, y_train_batch = pipeline('train').get_next()

logits_train_batch = forward(X_train_batch, reuse=False, is_training=True)

ops['global_step'] = tf.Variable(0, trainable=False)

ops['lr'] = tf.train.exponential_decay(5e-3, ops['global_step'], 1400, 0.2)

ops['loss'] = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train_batch, labels=y_train_batch))

ops['train'] = tf.train.AdamOptimizer(ops['lr']).apply_gradients(
    clip_grads(ops['loss']), global_step=ops['global_step'])

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
while True:
    try:
        sess.run(ops['train'])
    except tf.errors.OutOfRangeError:
        break
    else:
        step = sess.run(ops['global_step'])
        if step % DISPLAY_STEP == 0 or step == 1:
            loss, lr = sess.run([ops['loss'], ops['lr']])
            print("Step %d | Loss %.3f | LR: %.4f" % (step, loss, lr))

Step 1 | Loss 0.714 | LR: 0.0050
Step 50 | Loss 0.349 | LR: 0.0047
Step 100 | Loss 0.280 | LR: 0.0045
Step 150 | Loss 0.379 | LR: 0.0042
Step 200 | Loss 0.304 | LR: 0.0040
Step 250 | Loss 0.246 | LR: 0.0038
Step 300 | Loss 0.368 | LR: 0.0035
Step 350 | Loss 0.357 | LR: 0.0033
Step 400 | Loss 0.654 | LR: 0.0032
Step 450 | Loss 0.452 | LR: 0.0030
Step 500 | Loss 0.369 | LR: 0.0028
Step 550 | Loss 0.323 | LR: 0.0027
Step 600 | Loss 0.153 | LR: 0.0025
Step 650 | Loss 0.104 | LR: 0.0024
Step 700 | Loss 0.204 | LR: 0.0022
Step 750 | Loss 0.195 | LR: 0.0021
Step 800 | Loss 0.071 | LR: 0.0020
Step 850 | Loss 0.065 | LR: 0.0019
Step 900 | Loss 0.148 | LR: 0.0018
Step 950 | Loss 0.039 | LR: 0.0017
Step 1000 | Loss 0.139 | LR: 0.0016
Step 1050 | Loss 0.089 | LR: 0.0015
Step 1100 | Loss 0.053 | LR: 0.0014
Step 1150 | Loss 0.195 | LR: 0.0013
Step 1200 | Loss 0.003 | LR: 0.0013
Step 1250 | Loss 0.204 | LR: 0.0012
Step 1300 | Loss 0.109 | LR: 0.0011
Step 1350 | Loss 0.231 | LR: 0.0011
Step 1400 | Los

In [8]:
ops['predict'] = forward(pipeline('infer').get_next(), reuse=True, is_training=False)

In [9]:
y_pred_li = []
while True:
    try:
        y_pred_li.append(sess.run(ops['predict']))
    except tf.errors.OutOfRangeError:
        break
y_pred = np.argmax(np.vstack(y_pred_li), 1)
print("Accuracy: %.4f" % (y_pred==y_test).mean())
print(classification_report(y_test, y_pred))

Accuracy: 0.8980
             precision    recall  f1-score   support

          0       0.88      0.92      0.90     12500
          1       0.92      0.88      0.90     12500

avg / total       0.90      0.90      0.90     25000

